In [61]:
#Individual Homework 1
import pandas as pd
import requests
import io
import os

In [62]:
''' (START OF COMMENTING OUT - Attempted to use Classes but ran into several errors - will merge and clean data using regular fuctions instead.)
class WorldBankCleaning:
    def __init__(self,intermediate_folder="data/intermediate/", raw_folder="data/raw/"):
        self.intermediate_folder = intermediate_folder
        self.raw_folder = raw_folder
        self.ag_emp = None
        self.rural_growth = None
    
    def download_worldbank(indicator, countries, date_start, date_end):
        url_base = 'http://api.worldbank.org/v2/'  # Base URL for the World Bank API
        country_codes = ';'.join(countries)  # Combine country codes into a string
        url = url_base + f'country/{country_codes}/indicator/{indicator}?date={date_start}:{date_end}&per_page=30000' #create the url with start and end date.
        url = url_base + f'country/{country_codes}/indicator/{indicator}?per_page=30000' # This line overrides the previous one. It will ignore start/end date.

        response = requests.get(url)  # Download data from the URL
        df = pd.read_xml(response.content)  # Convert the downloaded data to a table
        return df  # Return the table
    
    def clean_wb(data):
        data = data.drop(["country","unit","decimal","obs_status","indicator"], axis=1)
        data = data.rename({"countryiso3code":'country',"date":'year'}, axis=1)
        return data
    
wb = WorldBankCleaning()
'''

' (START OF COMMENTING OUT - Attempted to use Classes but ran into several errors - will merge and clean data using regular fuctions instead.)\nclass WorldBankCleaning:\n    def __init__(self,intermediate_folder="data/intermediate/", raw_folder="data/raw/"):\n        self.intermediate_folder = intermediate_folder\n        self.raw_folder = raw_folder\n        self.ag_emp = None\n        self.rural_growth = None\n\n    def download_worldbank(indicator, countries, date_start, date_end):\n        url_base = \'http://api.worldbank.org/v2/\'  # Base URL for the World Bank API\n        country_codes = \';\'.join(countries)  # Combine country codes into a string\n        url = url_base + f\'country/{country_codes}/indicator/{indicator}?date={date_start}:{date_end}&per_page=30000\' #create the url with start and end date.\n        url = url_base + f\'country/{country_codes}/indicator/{indicator}?per_page=30000\' # This line overrides the previous one. It will ignore start/end date.\n\n        

In [63]:
''' ****************************************
*              FUNCTIONS                   *
*****************************************'''

def download_worldbank(indicator, countries, date_start, date_end):
    url_base = 'http://api.worldbank.org/v2/'  # Base URL for the World Bank API
    country_codes = ';'.join(countries)  # Combine country codes into a string
    url = url_base + f'country/{country_codes}/indicator/{indicator}?date={date_start}:{date_end}&per_page=30000' #create the url with start and end date.
    url = url_base + f'country/{country_codes}/indicator/{indicator}?per_page=30000' # This line overrides the previous one. It will ignore start/end date.

    response = requests.get(url)  # Download data from the URL
    df = pd.read_xml(response.content)  # Convert the downloaded data to a table
    return df  # Return the table

def clean_wb(data):
    data = data.drop(["country","unit","decimal","obs_status","indicator"], axis=1)
    data = data.rename({"countryiso3code":'country',"date":'year'}, axis=1)
    return data

def change_date(data):
    data.year = pd.PeriodIndex(data.year, freq = 'Q').to_timestamp()
    return data

In [64]:
#Variables for Future: 'SL.AGR.EMPL.ZS','SP.RUR.TOTL.ZG','SI.POV.NAHC', 'SI.POV.GINI','IT.NET.SECR.P6','IT.CEL.SETS.P2','FP.CPI.TOTL','FX.OWN.TOTL.ZS','EG.ELC.ACCS.ZS','EG.ELC.ACCS.RU.ZS','BX.TRF.PWKR.DT.GD.ZS'
ag_emp = download_worldbank(
    indicator = 'SL.AGR.EMPL.ZS',
    countries = ['BGD', 'IND','VNM','LAO','IND','THA','KHM','MYS','SGP','MMR','PHL','BRN'],
    date_start = '2020',
    date_end = '2023'
)

#'SL.AGR.EMPL.ZS','SP.RUR.TOTL.ZG','SI.POV.NAHC', 'SI.POV.GINI','IT.NET.SECR.P6','IT.CEL.SETS.P2','FP.CPI.TOTL','FX.OWN.TOTL.ZS','EG.ELC.ACCS.ZS','EG.ELC.ACCS.RU.ZS','BX.TRF.PWKR.DT.GD.ZS'
rural_growth = download_worldbank(
    indicator = 'SP.RUR.TOTL.ZG',
    countries = ['BGD', 'IND','VNM','LAO','IND','THA','KHM','MYS','SGP','MMR','PHL','BRN'],
    date_start = '2021',
    date_end = '2023'
)


In [65]:
ag_emp = clean_wb(ag_emp)

ag_emp = ag_emp.rename({'value':"Employment in Agr. (percent of total employment)"},axis=1)

ag_emp.head()

rural_growth = clean_wb(rural_growth)
rural_growth = rural_growth.rename({'value':"Rural Pop. Growth (annual percent)"},axis=1)

rural_growth.head()

,country,year,Rural Pop. Growth (annual percent)
0,BGD,2024,NaN
1,BGD,2023,-0.050253
2,BGD,2022,-0.236671
3,BGD,2021,-0.436696
4,BGD,2020,-0.404684


In [66]:
rural_growth = change_date(rural_growth)
ag_emp = change_date(ag_emp)

In [67]:
# (Checking Results)
rural_growth.year.dtype

rural_growth.head()

,country,year,Rural Pop. Growth (annual percent)
0,BGD,2024-01-01,NaN
1,BGD,2023-01-01,-0.050253
2,BGD,2022-01-01,-0.236671
3,BGD,2021-01-01,-0.436696
4,BGD,2020-01-01,-0.404684


In [68]:
ag_emp = ag_emp.set_index(['country','year'])
ag_emp.head()

rural_growth = rural_growth.set_index(['country','year'])
rural_growth.head()

Rural Pop. Growth (annual percent)
country year                                          
BGD     2024-01-01                                 NaN
        2023-01-01                           -0.050253
        2022-01-01                           -0.236671
        2021-01-01                           -0.436696
        2020-01-01                           -0.404684

In [69]:
wb_merged = pd.merge(
    rural_growth,
    ag_emp,
    right_index= True,
    left_index = True,
    how = 'inner'
)

wb_merged.head()

Rural Pop. Growth (annual percent)  \
country year                                             
BGD     2024-01-01                                 NaN   
        2023-01-01                           -0.050253   
        2022-01-01                           -0.236671   
        2021-01-01                           -0.436696   
        2020-01-01                           -0.404684   

                    Employment in Agr. (percent of total employment)  
country year                                                          
BGD     2024-01-01                                               NaN  
        2023-01-01                                         35.274072  
        2022-01-01                                         35.662164  
        2021-01-01                                         37.306117  
        2020-01-01                                         37.971569

In [ ]:
wb_merged.describe()

wb_merged.query("country in ['BGD', 'VNM']").describe()

,Rural Pop. Growth (annual percent),Employment in Agr. (percent of total employment)
count,630.000000,363.000000
mean,1.190278,39.995062
std,1.753955,25.074010
min,-8.817944,0.090952
25%,0.462129,15.628217
50%,1.327083,44.206308
75%,1.954488,60.592428
max,26.725026,86.474423
